In [1]:
# from websocket import create_connection
import websockets
from datetime import datetime
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import VotingClassifier
import pickle
from sklearn.metrics import accuracy_score
# from keras.models import load_model
# from keras.wrappers.scikit_learn import KerasClassifier

In [2]:
# model1 = load_model('../models/h5s/LSTM.h5')
# model1 = KerasClassifier(build_fn=model1)
# model1._estimator_type = "classifier"
# model2 = load_model('../models/h5s/GRU.h5')
# model2 = KerasClassifier(build_fn=model2)
# model2._estimator_type = "classifier"
model3 = pickle.load(open('../models/h5s/random-forest.h5', 'rb'))
model4 = pickle.load(open('../models/h5s/logistic-regression.h5', 'rb'))
model5 = pickle.load(open('../models/h5s/linear-discriminant-analysis.h5', 'rb'))
model6 = pickle.load(open('../models/h5s/kNN.h5', 'rb'))
model7 = pickle.load(open('../models/h5s/CART.h5', 'rb'))
model8 = pickle.load(open('../models/h5s/support-vector-machine.h5', 'rb'))
model9 = pickle.load(open('../models/h5s/naive-bayes.h5', 'rb'))

In [3]:
def log(content, new=False, file='stream.log', timestamp=False, print_line=0):
    if(new):
        log = open(f'./{file}', 'w')
        log.write("")
        log.close()
    log = open(f'./{file}', 'a')

    if(timestamp):
        log.write(f"\n{datetime.now()} ~ {content}")
    else:
        log.write(content)
    
    if(print_line):
        line = "-" * print_line
        log.write(f"\n{line}\n\n")
    log.close()


In [4]:
async def ensemble(IoT, chunk_count, initial=False):
    x = IoT.drop(['label', 'date', 'time'], axis=1)
    y = IoT['label']
    x_train,x_test,y_train,y_test = train_test_split(x, y, test_size=0.2)
 
    voting = VotingClassifier(estimators=[('SVM', model8), ('LR', model4), ('RF', model3), ('LDA', model5), ('KNN', model6), ('CART', model7), ('Naive-Bayes', model9)], voting='hard')
    voting.fit(x_train, y_train) 
    if(not initial):
        log(f"", file="ensemble.log", print_line=20)

    log(f"Model v{chunk_count}\n", initial, file="ensemble.log", timestamp=True)

    for clf in (model8, model4, model3, model5, model6, model7, model9, voting):
        clf.fit(x_train, y_train)
        y_pred = clf.predict(x_test)
        log(f"{clf.__class__.__name__}: {accuracy_score(y_test, y_pred)}\n", file="ensemble.log")

    return voting

In [5]:
INITIAL_CHUNK_SIZE = 50
FINAL_CHUNK_SIZE = 20
CHUNK_SIZE_INCREMENT_FACTOR = 2

async def client():
    uri = "ws://localhost:8765"
    temp_row_count = 0
    chunk_size = INITIAL_CHUNK_SIZE
    chunk_count = 0
    initial_chunk = True
    model = 0
    log("", True)
    async with websockets.connect(uri, ping_interval=None) as websocket:
        df = pd.DataFrame()
        while(True):
            row = await websocket.recv()
            log(f"{temp_row_count} ")
            data_list = row.split(",")

            if(temp_row_count == 0):
                df = pd.DataFrame()
                df = pd.DataFrame([data_list], columns = ["date", "time", "state", "sphone_signal", "label", "type"])
            else:
                df = pd.concat([df, pd.DataFrame([data_list], columns = ["date", "time", "state", "sphone_signal", "label", "type"])])
            temp_row_count += 1
            
            if (temp_row_count == chunk_size):
                if(initial_chunk):
                    try:
                        model = await ensemble(df, chunk_count+1, initial=True)
                        initial_chunk = False
                        chunk_count += 1
                        log(f"Chunk-{chunk_count} created with {temp_row_count} records.", timestamp=True, print_line=50)
                        chunk_size = FINAL_CHUNK_SIZE
                        temp_row_count = 0
                    except ValueError:
                        # print(sys.exc_info())
                        log("CLASS ERROR\n", timestamp=True)
                        chunk_size *= CHUNK_SIZE_INCREMENT_FACTOR
                else:
                    try:
                        model1 = await ensemble(df, chunk_count+1)
                        ensemble_model = VotingClassifier(estimators=[('New Model', model1), ('Old Model', model)], voting='hard')
                        # filename = 'model/ensmeble_model.h5'
                        # pickle.dump(ensemble_model, open(filename, 'wb'))
                        model = ensemble_model
                        chunk_count += 1
                        log(f"Chunk-{chunk_count} created with {temp_row_count} records.", timestamp=True, print_line=50)
                        chunk_size = FINAL_CHUNK_SIZE
                        temp_row_count = 0
                    except ValueError:
                        # print(sys.exc_info())
                        log("CLASS ERROR\n", timestamp=True)
                        chunk_size *= CHUNK_SIZE_INCREMENT_FACTOR

            await websocket.send("1")

await client() 

ConnectionClosedOK: received 1001 (going away); then sent 1001 (going away)